# Gas Dosing Demo

I created a makefile to make interacting with the Demo easier: 

Start Docker containers with the SEC Nodes running on port 10800 (gas_dosing) and 10801 (reactorcell):

 ``` make sim```

Start Containers and additionally start frappy gui:

```make frappy``` 

In [1]:
from bluesky import RunEngine
import bluesky.plan_stubs as bps
from bluesky.plan_stubs import sleep, rd
from bluesky.plans import scan, count

import databroker

from bluesky.preprocessors import run_decorator, SupplementalData
from bluesky import preprocessors as bpp
from bluesky.callbacks.best_effort import BestEffortCallback
from pprint import pprint
from secop_ophyd.SECoPDevices import SECoPNodeDevice
from bluesky.utils import ProgressBarManager

from tiled.client import from_uri


import time
from ophyd.status import Status


from bluesky.log import config_bluesky_logging

config_bluesky_logging(level='WARNING')
# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({},call_returns_result=True)
bec = BestEffortCallback()
RE.subscribe(bec)
RE.waiting_hook = ProgressBarManager()
RE.ignore_callback_exceptions = False



#Example of adding metadata to RE environment
investigation_id = "Nexus Demonstrator"

RE.md["investigation_id"] = investigation_id


client = from_uri("http://localhost:8000",api_key="secret")

def post_document(name,doc):
    client.post_document(name, doc)
    
RE.subscribe(post_document)

# Connect to Gas Dosing SEC Node and generate ophyd device tree
gas_dosing =  SECoPNodeDevice.create('localhost','10800',RE.loop)

# Connect to Reactor Cell SEC Node and generate ophyd device tree
reactor_cell =  SECoPNodeDevice.create('localhost','10801',RE.loop)

gas_dosing.class_from_instance()
reactor_cell.class_from_instance()


from genNodeClass import *

gas_dosing:Gas_dosing = gas_dosing
reactor_cell:Reactor_cell = reactor_cell



gas_dosing ready
reactor_cell ready
no code generated yet, building from scratch


### Using the Devices
From here on out the SECoP-Ophyd devices are fully usable and behave like any other ophyd-async device. They can be incorporated into bluesky plans alongside other devices, that might have a different underlying field layer such as EPICS or TANGO.   

In [ ]:
# Example of a scan plan:


def test_plan():
    yield from bps.abs_set(gas_dosing.backpressure_contr1,1000,wait=False,group='start')
    yield from bps.abs_set(reactor_cell.temperature_reg,300,wait=False,group='start')
    yield from bps.abs_set(gas_dosing.massflow_contr1,0,wait=False,group='start')
    yield from bps.abs_set(gas_dosing.massflow_contr2,0,wait=False,group='start')
    yield from bps.abs_set(gas_dosing.massflow_contr3,0,wait=False,group='start')

    yield from bps.wait(group='start')
    



    yield from bps.abs_set(gas_dosing.backpressure_contr1,900,wait=False)
    yield from bps.abs_set(gas_dosing.massflow_contr1,20,wait=False)
    yield from bps.abs_set(gas_dosing.massflow_contr2,30,wait=False)
    yield from bps.abs_set(gas_dosing.massflow_contr3,50,wait=False)
    yield from scan([
            reactor_cell.temperature_sam,
            gas_dosing.backpressure_contr1,
            gas_dosing.massflow_contr1,
            gas_dosing.massflow_contr2,
            gas_dosing.massflow_contr3
        ],
        reactor_cell.temperature_reg,300,310,10)


RE(test_plan())

In [4]:
import NexusCreator

variableDictionary = {
  'a': 1,
  'b': 2,
  'c': 3,
  'variable': [1, 2, 3],
  'image': [[1, 2, 3], [4, 5,6]]
}

flags = {
  'nxstruct': './nexuscreator/examples/test2.nxstruct',
  'input': variableDictionary,
  'dictionary': False
}

NexusCreator.executeConversion(flags)


In [4]:
import numpy as np
import NexusCreator
def flatten_structure(data, parent_key='', sep='.'):
    flattened = {}
    for key, value in data.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key

        if value != None:
            if isinstance(value, dict):
                flattened.update(flatten_structure(value, new_key, sep=sep))
            else:
                flattened[new_key] = value

    return flattened




# Connecting to Tiled
db = from_uri("http://136.244.80.119:8000",api_key="secret")
run = db['716034b0-ba80-4975-aa97-7e0dbe8626af']
#run.metadata['start']
#db[-1].descriptors

# Getting data + metadata from Tiled
# Getting metadata (settings of instruments before a scan) from Tiled
variables = flatten_structure(run.metadata['start']['nexus_md'])
# Getting data (measurement from scan) from Tiled
for key in run.primary.data.keys():
    variables[key] = np.asarray(run.primary.data.get(key)).tolist()

# Display the result
print(variables)

# Preparing NexusCreator
flags = {
  'nxstruct': './test2.nxstruct',
  'input': variables,
  'dictionary': False
}


{'aperture_1.world_position.x': 0, 'aperture_1.world_position.y': 9, 'aperture_1.world_position.z': 0, 'aperture_1.top': 100.0, 'aperture_1.bottom': 110.0, 'aperture_1.left': 120.0, 'aperture_1.right': 130.0, 'mirror_1.world_position': [0, 10, 0], 'mirror_1.coating': 'Pt', 'mirror_1.incident_angle': 1.5, 'mirror_1.tx': 10.0, 'mirror_1.ty': 30.0, 'mirror_1.rx': 0.0, 'mirror_1.ry': 40.0, 'mirror_1.rz': 20.0, 'motors_pgm.world_position': [0, 20, 0], 'motors_pgm.en': 400.0, 'motors_pgm.cff': 2.25, 'mirror_3.world_position': [0, 25, 0], 'mirror_3.coating': 'Pt', 'mirror_3.incident_angle': 1.5, 'mirror_3.tx': 15.0, 'mirror_3.ty': 35.0, 'mirror_3.tz': 35.0, 'mirror_3.rx': 5.0, 'mirror_3.ry': 45.0, 'mirror_3.rz': 25.0, 'exit_slit.world_position': [0, 50, 0], 'mirror_4.world_position': [0, 55, 0], 'mirror_4.tx': 14.0, 'mirror_4.ty': 34.0, 'mirror_4.tz': 34.0, 'mirror_4.rx': 4.0, 'mirror_4.ry': 44.0, 'mirror_4.rz': 24.0, 'detector_kth00.world_position': [0, 60, 0], 'detector_kth00.signal_source'

In [9]:
async def generate_nexus_struct(nodes:list[SECoPNodeDevice]):
    text = """
entry:NXentry
\t@NX_class = "NXentry"
\tsample:NXsample
\t\t@NX_class = "NXsample"
\t\ttype:sample environment
\t\t\t@NX_class = "NX_CHAR"
"""

    for node in nodes:
        equpment_i = 



    return text

generated_text = generate_nexus_struct(nodes= [gas_dosing])

print(generated_text)


entry:NXentry
	@NX_class = "NXentry"
	sample:NXsample
		@NX_class = "NXsample"
		type:sample environment
			@NX_class = "NX_CHAR"

